In [1]:
#Problem statement: Her Majesty the Queen normally only adresses the nation once every year,
#but due to the Corona-crisis, she has given given 3 public addresses during the last 4 months:
#- Traditional New Year's Speech 31st of December 2019
#- Speech on the Corona-situation 17th of March 2020
#- Speech on the occation of Her Majesty's 80th birthday 16th of April 2020

#Research question: Build a model that takes as input random individual sentences from one of the 3 speeches
#and attempts to predict which of the 3 speeches the sentence belongs to

In [2]:
import requests
import collections
import nltk
nltk.download("punkt")
import numpy as np
import pandas as pd
import sklearn.feature_extraction.text
import sklearn.metrics
import sklearn.model_selection
import sklearn.naive_bayes
import nltk
import requests
import bs4

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\timmo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
#Scrape the New Year's speech from www.kongehuset.dk and convert with BeautifulSoup plus some manual corrections
#Final format will be one long string

new_year = requests.get('http://kongehuset.dk/nyheder/laes-hm-dronningens-nytaarstale-2019')
new_year.raise_for_status()
new_year_soup = bs4.BeautifulSoup(new_year.text, parser='html')
new_year_elements = new_year_soup.find_all('p')
new_year_string = str(new_year_elements)

def clean_soup(stringo, clean_list):
    for x in clean_list:
        stringo = stringo.replace(x, '')
    return stringo

clean_listo = ['<p>', '</p>', '//', '<p class="rtecenter">', '<span>', '|Nyhed', '</span>', '<br/>', '31. december 2019', '[', ']']

new_year_string = clean_soup(new_year_string, clean_listo)
new_year_string = new_year_string[27:]
print(new_year_string)


 Så er endnu et år gået. Nu ligger 2019 bag os med alt hvad det bragte: - minder vi vil hæfte os ved, både de gode og de triste, - udfordringer, som vi måtte tage op, både de tunge og de inspirerende., I år var det 50 år siden, at mennesket landede på månen og vi fik vores egen planet Jorden at se som en lille klode i det store rum: ganske alene, men så smuk og rund og blå: Planeten, hvor vi har hjemme. For os her i Danmark er det måske ikke så overraskende, at planeten er blå, for vi har jo havet foran os og den blå himmel over os., Så storslået og varieret vor Jord end kan synes, er den dog sårbar. Det er vi ved at lære at indse, og det kan godt bekymre, ikke mindst mange unge, som ser klimaforandringerne, der gør sig tydeligt gældende i disse år. Vi har en fælles forpligtelse for vores smukke klode, så myldrende fuld af liv. Det er en væsentlig udfordring for os alle i dag, og det gælder om, at vi alle er opmærksomme på, hvordan vi lever og hvad vi gør., I vores tid har der sneget s

In [4]:
#Do the same for the Corona-speech and add to the clean_list as required

corona = requests.get('http://kongehuset.dk/nyheder/laes-hm-dronningens-tale-til-befolkningen')
corona.raise_for_status()
corona_soup = bs4.BeautifulSoup(corona.text, parser='html')
corona_elements = corona_soup.find_all('p')
corona_string = str(corona_elements)
clean_listo = clean_listo + ['<a href=', '"http:kongehuset.dk/presse"', ' tabindex="0">Pressefotos</a>']
corona_string = clean_soup(corona_string, clean_listo)
corona_string = corona_string[41:]

print(corona_string)

#We note that the final sentence is 'Billeder kan downloades til redaktionel brug' and will remember to correct that later

 Danmark står i en alvorlig situation. Den skæbne deler vi med hele Europa, ja, med resten af verden.

Mange mennesker er blevet ramt af corona-virus, og vi ved, at mange flere vil blive syge i den nærmeste tid. , Jeg kan godt forstå, at mange er urolige og bekymrede, for vores virkelighed og vores hverdag er blevet vendt på hovedet. , Vi har vænnet os til, at verden ligger åben, nu er grænserne lukket. , Foråret er på vej, Danmark sprudlede af liv. Nu er dagligdagen sat i stå. Det føles både skræmmende og uvirkeligt, men det er vores nye virkelighed og vi må indse og lære, at det kræver noget af os alle sammen. , Myndighederne har igennem den sidste uges tid været nødt til at træffe en række nødvendige beslutninger, som vi alle kan mærke i vores dagligdag. , Det angår os alle sammen. For lidt siden har Statsministeren været nødt til at komme med endnu stærkere beslutninger. Vi må gøre noget hver især., Hvad vi gør, og hvordan vi handler i disse dage, kan blive afgørende for, hvordan s

In [5]:
birthday = requests.get('http://kongehuset.dk/nyheder/laes-hm-dronningens-tale-i-anledning-af-80-aars-foedselsdagen')
birthday.raise_for_status()
birthday_soup = bs4.BeautifulSoup(birthday.text, parser='html')
birthday_elements = birthday_soup.find_all('p')
birthday_string = str(birthday_elements)
birthday_string = clean_soup(birthday_string, clean_listo)
birthday_string = birthday_string[41:]
print(birthday_string)

 At fejre fødselsdag er en dyb og rodfæstet tradition i Danmark. Vi fejrer den forskelligt, men for de fleste skal det helst være sammen med familie og venner., Sådan plejer det også at være i min familie. Jeg har altid glædet mig over at kunne fejre min fødselsdag. At kunne mærke, helt bogstaveligt, den hjertevarme stemning, der strømmer mig i møde på selve dagen. Det har altid betydet noget ganske særligt for mig., Sådan har det ikke været i år., Vi har fået besøg af en ubuden og farlig gæst, som har sat sit præg på hele landet. Mange fester, konfirmationer og bryllupper er blevet berørt, og sådan er det også med min fødselsdag., Har det så været en lang og trist dag?, Nej, slet ikke. Dagen har tværtimod bragt mig så mange glæder, og har beriget mig, mere end jeg kan sige., Det rører mig dybt, at så mange har ønsket at være med til at fejre mig også i år. Jeg takker af hele mit hjerte for hilsenerne, for sangene, og for de mange tanker, som i dagens løb er strømmet mig i møde fra all

In [6]:
#To understand whether the 3 speeches are really that different, we will first describe:
#- The number of words
#- The number of unique words
#- All bigrams that occur more than once
#- All trigrams that occur more than once
 
def analyze(stringo):
    #Return a list where the first element is number of words, second element is number of unique words, 
    #third element is all bigrams that occur more than once, fourth element is all trigrams that occur more than once
    result_list = []
    #analyze number of unique words
    stringo_1 = stringo.replace(',', '')
    stringo_1 = stringo_1.replace('.', '')
    stringo_1 = stringo_1.lower()
    listo_1 = stringo_1.split()
    result_list.append(str(len(stringo_1)))
        
    unique = []
    for x in listo_1:
        if not x in unique:
            unique.append(x)
    result_list.append(str(len(unique)))
        
    tim_bigrams = list(nltk.bigrams(listo_1))  
    bigram_unique = []
    for x in tim_bigrams:
        if x not in bigram_unique:
            bigram_unique.append(x)
    bigram_count_dict = {}
    for x in bigram_unique:
        count = 0
        for y in tim_bigrams:
            if x == y:
                count += 1
        bigram_count_dict[x] = count
    bigram_count_df = pd.DataFrame.from_dict(bigram_count_dict, columns=['Count'], orient='index')
    bigram_count_df = bigram_count_df.sort_values(by=['Count'], ascending=False)
    bigram_count_df = bigram_count_df[bigram_count_df['Count'] > 3]
    result_list.append(str(bigram_count_df.index))
        
    tim_trigrams = list(nltk.ngrams(listo_1, 3))
    trigram_unique = []
    for x in tim_trigrams:
        if x not in trigram_unique:
            trigram_unique.append(x)
    trigram_count_dict = {}
    for x in trigram_unique:
        count = 0
        for y in tim_trigrams:
            if x == y:
                count += 1
        trigram_count_dict[x] = count
    trigram_count_df = pd.DataFrame.from_dict(trigram_count_dict, columns=['Count'], orient='index')
    trigram_count_df = trigram_count_df.sort_values(by=['Count'], ascending=False)
    trigram_count_df = trigram_count_df[trigram_count_df['Count'] > 2]
    result_list.append(str(trigram_count_df.index))
    
    return result_list

new_year_result = analyze(new_year_string)
corona_result = analyze(corona_string)
birthday_result = analyze(birthday_string)

def show_clean(keys_0):
    keys_0 = str(keys_0)
    keys_0 = keys_0[8:-17]
    return keys_0

print('DESCRIPTIVE STATISTICS')
print('Number of words:')
print('New Year: ' + str(new_year_result[0]))
print('Corona: ' + str(corona_result[0]))
print('Birthday: ' + str(birthday_result[0]))
print()
print('Number of unique words:')
print('New Year: ' + str(new_year_result[1]))
print('Corona: ' + str(corona_result[1]))
print('Birthday: ' + str(birthday_result[1]))
print()
print('Number of bigrams ocurring more than 3 times:')
print('New Year: ' + show_clean(new_year_result[2]))
print('Corona: ' + show_clean(corona_result[2]))
print('Birthday: ' + show_clean(birthday_result[2]))
print()
print('Number of trigrams ocurring more than once:')
print('New Year: ' + show_clean(new_year_result[3]))
print('Corona: ' + show_clean(corona_result[3]))
print('Birthday: ' + show_clean(birthday_result[3]))

DESCRIPTIVE STATISTICS
Number of words:
New Year: 6595
Corona: 3368
Birthday: 3544

Number of unique words:
New Year: 496
Corona: 291
Birthday: 288

Number of bigrams ocurring more than 3 times:
New Year:  ('det', 'er'),   ('vi', 'har'),   ('og', 'det'),   ('for', 'at'),
        ('har', 'jeg'),   ('og', 'som'), ('til', 'alle'),   ('er', 'det'),
           ('er', 'i')],
    
Corona: 'det', 'er')]
Birthday: 'til', 'at'), ('vi', 'har'), ('at', 'være')]

Number of trigrams ocurring more than once:
New Year: 'det', 'nye', 'år'), ('vi', 'har', 'brug'), ('har', 'brug', 'for')]
Corona: 'os', 'alle', 'sammen')]
Birthday: 


In [7]:
#MODELLING: we now build 3 separate dataframes with a new Event = New Year, Corona or Birthday
#Build 3 separate dataframes that have one column with each sentence and one column designating the event
new_year_df = pd.DataFrame(new_year_string.split('.'), columns=['Text'], dtype='string')
new_year_df['Event'] = 'New Year'

corona_df = pd.DataFrame(corona_string.split('.'), columns=['Text'], dtype='string')
corona_df['Event'] = 'Corona'
corona_df = corona_df.drop([48, 49])

birthday_df = pd.DataFrame(birthday_string.split('.'), columns=['Text'], dtype='string')
birthday_df['Event'] = 'Birthday'

#We then combine the 3 dataframes into one 

speech_df = pd.concat([new_year_df, corona_df, birthday_df], ignore_index=True)

def count_unique(series):
    count_words = 0

    unique = []
    for x in series:
        x1 = x.split()
        for y in x1:
            count_words += 1
            if not y in unique:
                unique.append(y)
    unique.sort()
    print('This is the number of words: ' + str(count_words))

    print('This is the number of unique: ' + str(len(unique)))

speech_df



,Text,Event
0,Så er endnu et år gået,New Year
1,Nu ligger 2019 bag os med alt hvad det bragte...,New Year
2,", I år var det 50 år siden, at mennesket lande...",New Year
3,For os her i Danmark er det måske ikke så ove...,New Year
4,", Så storslået og varieret vor Jord end kan sy...",New Year
...,...,...
165,"Så kan vi vende tilbage til et Danmark, som n...",Birthday
166,", Min fødselsdag blev ikke, som jeg havde for...",Birthday
167,", Nu åbner vi Danmark langsomt igen",Birthday
168,Mine tanker og hilsener går på ny til hver en...,Birthday


In [8]:
count_unique(speech_df['Text'])

This is the number of words: 2600
This is the number of unique: 930


In [9]:
#FIRST MODEL: 3 events, remove general stopwords in Danish language rather than stopwords that are specific to the Queen
#A total of 789 unique words after removing stopwords

#https://gist.github.com/berteltorp/0cf8a0c7afea7f25ed754f24cfc2467b#file-stopord-txt

speech_df_1 = speech_df.copy()

stop_word_file = open('stopord.txt', encoding='latin1')
stop_word_text = stop_word_file.read()

def remove_stop_words(text):
    text = text.split()
    text_2 = ''
    for x in text:
        if not x in stop_word_text:
            text_2 = text_2 + x + ' '
    return text_2

speech_df_1['Text'] = speech_df_1['Text'].map(remove_stop_words)

count_unique(speech_df_1['Text'])

This is the number of words: 1356
This is the number of unique: 789


In [10]:
#generate a bag of words, i.e. an array for each sentence showing the unique words that appear in the sentence
tim_vectorizer = sklearn.feature_extraction.text.CountVectorizer()

tim_bow_1 = tim_vectorizer.fit_transform(speech_df_1['Text'])

tim_bow_array_1 = tim_bow_1.toarray()

tim_vectorizer.get_feature_names()

['10',
 '100',
 '1945',
 '2019',
 '2020',
 '50',
 '80',
 'af',
 'afgørende',
 'afskåret',
 'afstand',
 'aften',
 'aldre',
 'alene',
 'alle',
 'alles',
 'allesammen',
 'alt',
 'alvoren',
 'alvorlig',
 'alvorlige',
 'alvorligt',
 'anderledes',
 'andre',
 'angår',
 'ansigt',
 'ansvar',
 'antisemitisme',
 'antisemitismen',
 'anvisninger',
 'appel',
 'april',
 'arbejde',
 'argentina',
 'arv',
 'at',
 'barn',
 'bedstemor',
 'befinder',
 'befolkning',
 'befrielsen',
 'begivenhed',
 'bekendt',
 'bekendtskab',
 'bekræfte',
 'bekræftet',
 'bekymre',
 'bekymrede',
 'bekymringer',
 'beredskabet',
 'beriget',
 'berusende',
 'berørt',
 'besat',
 'beskæmmende',
 'beslutninger',
 'bestille',
 'besværligt',
 'besøg',
 'besøgte',
 'betro',
 'betyder',
 'betydet',
 'bevare',
 'bevidsthed',
 'bidrager',
 'bidraget',
 'blad',
 'blevet',
 'bliv',
 'blot',
 'blå',
 'bogstaveligt',
 'bombardement',
 'bor',
 'bornholm',
 'bragt',
 'bragte',
 'bringe',
 'bringer',
 'brug',
 'bryde',
 'bryllupper',
 'budskab',
 

In [12]:
#Define a model that uses the bag of words for each sentence to predict the 'Event'-category parameter
#I have a small data set of 173 observations so I select a high test_size = 0.2

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(tim_bow_array_1, speech_df_1['Event'], test_size = 0.2)

tim_model = sklearn.naive_bayes.MultinomialNB()

tim_model.fit(X_train, y_train)

print(sklearn.metrics.classification_report(y_test, tim_model.predict(X_test)))

              precision    recall  f1-score   support

    Birthday       0.44      0.67      0.53         6
      Corona       0.50      0.45      0.48        11
    New Year       0.73      0.65      0.69        17

    accuracy                           0.59        34
   macro avg       0.56      0.59      0.57        34
weighted avg       0.61      0.59      0.59        34



In [24]:
#Accuracy = number of correct predictions / total number of predictions
#Precision = true positive / true positives and false positives
#- so every time I predicted that this sentence was from the birthday speech, how often was I actually correct
#Recall = true positive / true positives and false negatives
#- so every time a sentence from the birthday speech ocurred in the data, how often was I able to predict it correctly
#f1-score = a combination of Precision and Recall

#A simple model would be right 33% of the time or 50% if we take into account that the New Year's speech was
#twice as long as the other two speeches, so these numbers are not very impressive


In [13]:
#SECOND MODEL: From 3 event categories to two
#New Year = Traditional and Corona and Birthday are combined into one category = Exceptional

speech_df_2 = speech_df_1.copy()

speech_df_2['Combined Event'] = 'X'

speech_df_2['Combined Event'][speech_df_2['Event'] == 'New Year'] = 'Traditional'

speech_df_2['Combined Event'][speech_df_2['Event'] == 'Corona'] = 'Exceptional'

speech_df_2['Combined Event'][speech_df_2['Event'] == 'Birthday'] = 'Exceptional'

speech_df_2

,Text,Event,Combined Event
0,Så år gået,New Year,Traditional
1,"Nu ligger 2019 bragte: - minder hæfte ved, båd...",New Year,Traditional
2,", I år 50 år siden, mennesket landede på månen...",New Year,Traditional
3,"For Danmark måske så overraskende, planeten bl...",New Year,Traditional
4,", Så storslået varieret Jord synes, sårbar",New Year,Traditional
...,...,...,...
165,"Så vende Danmark, være forandret, være alles D...",Birthday,Exceptional
166,", Min fødselsdag ikke, forestillet mig, taknem...",Birthday,Exceptional
167,", Nu åbner Danmark langsomt",Birthday,Exceptional
168,Mine tanker hilsener går på enkelt landet hele...,Birthday,Exceptional


In [14]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(tim_bow_array_1, speech_df_2['Combined Event'], test_size = 0.2)

tim_model.fit(X_train, y_train)

print(sklearn.metrics.classification_report(y_test, tim_model.predict(X_test)))

#This model performs better than the default 50%, although not impressive
#I also try dropping the Birthday-speech altoghether, but the result is about the same

              precision    recall  f1-score   support

 Exceptional       0.88      0.61      0.72        23
 Traditional       0.50      0.82      0.62        11

    accuracy                           0.68        34
   macro avg       0.69      0.71      0.67        34
weighted avg       0.75      0.68      0.69        34



In [15]:
#MODEL 3:Keep the two event categories and remove more stopwords

#Jens-Erik suggested to look at removing more stopwords. 
#I therefore make a new iteration where I do not use the general stopword-list from Github but
#generate a stopword-list from the speeches and set a high cut-off.
#If the original 3 texts had 930 unique words and an application of a general stopword-list 
#reduced the number to 789, I could remove the 200 most common stopwords

speech_df_3 = speech_df.copy()

speech_df_3['Combined Event'] = 'X'

speech_df_3['Combined Event'][speech_df_3['Event'] == 'New Year'] = 'Traditional'

speech_df_3['Combined Event'][speech_df_3['Event'] == 'Corona'] = 'Exceptional'

speech_df_3['Combined Event'][speech_df_3['Event'] == 'Birthday'] = 'Exceptional'


def get_stopwords(dataframe, column, n_most_common):
    counter = collections.Counter()
    dataframe[column].str.lower().str.split().apply(counter.update)
    return [word for word, _ in counter.most_common(n_most_common)]

stop_word_text = get_stopwords(speech_df_3, 'Text', 200)

speech_df_3['Text'] = speech_df_3['Text'].str.lower().map(remove_stop_words)

compare_df = pd.concat([speech_df['Text'], speech_df_3['Text']], axis=1)

compare_df


C:\Users\timmo\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\timmo\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\timmo\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Text,Text
0,Så er endnu et år gået,gået
1,Nu ligger 2019 bag os med alt hvad det bragte...,"2019 alt bragte: hæfte triste, udfordringer, o..."
2,", I år var det 50 år siden, at mennesket lande...",50 mennesket landede månen egen planet jorden ...
3,For os her i Danmark er det måske ikke så ove...,"overraskende, planeten blå, havet blå himmel"
4,", Så storslået og varieret vor Jord end kan sy...","storslået varieret vor jord synes, sårbar"
...,...,...
165,"Så kan vi vende tilbage til et Danmark, som n...","vende tilbage danmark, nok forandret, alligeve..."
166,", Min fødselsdag blev ikke, som jeg havde for...","havde forestillet taknemmelig den, bidraget gø..."
167,", Nu åbner vi Danmark langsomt igen",åbner langsomt
168,Mine tanker og hilsener går på ny til hver en...,hilsener ny enkelt riget


In [16]:
tim_bow_3 = tim_vectorizer.fit_transform(speech_df_3['Text'])

tim_bow_array_3 = tim_bow_3.toarray()

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(tim_bow_array_3, speech_df_3['Combined Event'], test_size = 0.2)

tim_model.fit(X_train, y_train)

print(sklearn.metrics.classification_report(y_test, tim_model.predict(X_test)))

#By running various simulations, I see that the biggest improvement comes from 
#calculating the stopwords directly, not by increasing their number
#also if I increase the cut-off for stopwords further, I lose entire sentences
#so fewer observations

              precision    recall  f1-score   support

 Exceptional       0.60      0.64      0.62        14
 Traditional       0.74      0.70      0.72        20

    accuracy                           0.68        34
   macro avg       0.67      0.67      0.67        34
weighted avg       0.68      0.68      0.68        34



In [ ]:
#MODEL 3 IS THEREFORE THE SELECTED MODEL, FIT IS NOT IMPRESSIVE BUT BETTER THAN THE DEFAULT 50%
#It seems to predict better on exceptional than on traditional 

#Reflections on fairness:

#This model attempts to predict the timing of public speeches made by one public figure
#If the modelling is succesful (which it was not really here), it could be used to assess
#How people's moods - as expressed by their social media activity - is affected by
#life events such as loss of employement or divorce


In [17]:
#Now for party purposes, build a simulator that takes the 3 speeches as input
#and outputs a simulated Queen's speech of a given length  
#This will include stopwords so we work on the original speech_df dataframe, column 'Text'

total_speech = ''

speech_df_4 = speech_df.copy()

speech_df_4['Text'] = speech_df_4['Text'].str.lower()

for i in speech_df_4.index[:-1]:
    texto = speech_df_4['Text'].iloc[i]
    if not 'Gud bevare Danmark' in texto:
        total_speech = total_speech + ' ' + texto

tim_bigrams = nltk.bigrams(total_speech.split())

tim_bigram_counter ={}

for x in tim_bigrams:
    w1, w2 = x
    if w1 not in tim_bigram_counter:
        tim_bigram_counter[w1] = {}
    if w2 not in tim_bigram_counter[w1]:
        tim_bigram_counter[w1][w2] = 1
    else:
        tim_bigram_counter[w1][w2] += 1   

In [18]:
def calculate_probability_distribution(bigram_counter):
    for key, values in bigram_counter.items():
        total = sum(values.values())
        n = {}
        for word, count in values.items():
            n[word] = count/total
            bigram_counter[key] = n
    return bigram_counter
tim_bigram_counter = calculate_probability_distribution(tim_bigram_counter)

In [22]:
sentence = ['danmark']
for i in range(100):
    choices = [k for k in tim_bigram_counter[sentence[-1]].keys()]
    probs = [p for p in tim_bigram_counter[sentence[-1]].values()]
    sentence.append(np.random.choice(choices, p=probs))
print(" ".join(sentence), end='')
print('. Gud bevare Danmark.')

danmark – rodfæstet i fællesskabet - ikke være med ønsket om danmarks historie på færøerne går til alle sammen , jeg slesvig-holsten, hvor prins joachim efteruddanner sig i år sammen med at vi skal prins christian konfirmeres han ville være god til , hver enkelt her i stort og som føler sig for alle på denne aften, hvor blev også her hos os sammen med alt hvad det 5 år , hvad det rørte mig , den nærmeste uger , 2020 bliver et blad sig i grønland , prins joachim efteruddanner sig tid til alle, der kom befrielsen den 4 og. Gud bevare Danmark.
